In [1]:
import os
import pickle
import argparse

import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres

from hpbandster.optimizers import BOHB

import import_ipynb
from bohb_worker_1to3_11 import KerasWorker as worker

parser = argparse.ArgumentParser(description='bohb_server')
parser.add_argument('--min_budget',   type=float, help='Minimum number of epochs for training.',    default=50)
parser.add_argument('--max_budget',   type=float, help='Maximum number of epochs for training.',    default=200)
parser.add_argument('--n_iterations', type=int,   help='Number of iterations performed by the optimizer', default=100)
parser.add_argument('--worker', help='Flag to turn this into a worker process', action='store_true')
parser.add_argument('--run_id', type=str, help='A unique run id for this optimization run. An easy option is to use the job id of the clusters scheduler.')
parser.add_argument('--nic_name',type=str, help='Which network interface to use for communication.', default='lo')
parser.add_argument('--shared_directory',type=str, help='A directory that is accessible for all processes, e.g. a NFS share.', default='../BOHB_LAB_5CNN/result/11')

args=parser.parse_args(args=[])
    
# Every process has to lookup the hostname
host = hpns.nic_name_to_host(args.nic_name)


if args.worker:
    import time
    time.sleep(5)   # short artificial delay to make sure the nameserver is already running
    w = worker(run_id=args.run_id, host=host, timeout=120)
    w.load_nameserver_credentials(working_directory=args.shared_directory)
    w.run(background=False)
    exit(0)

# This example shows how to log live results. This is most useful
# for really long runs, where intermediate results could already be
# interesting. The core.result submodule contains the functionality to
# read the two generated files (results.json and configs.json) and
# create a Result object.
result_logger = hpres.json_result_logger(directory=args.shared_directory, overwrite=False)

# Start a nameserver:
NS = hpns.NameServer(run_id=args.run_id, host=host, port=0, working_directory=args.shared_directory)
ns_host, ns_port = NS.start()
# NS = hpns.NameServer(run_id=args.run_id, host='127.0.0.1', port=None, working_directory=args.shared_directory)
# ns_host, ns_port = NS.start()

# Start local worker
w = worker(run_id=args.run_id, host=host, nameserver=ns_host, nameserver_port=ns_port, timeout=120)
w.run(background=True)

# Run an optimizer
bohb = BOHB(  configspace = worker.get_configspace(),
                      run_id = args.run_id,
                      host=host,
                      nameserver=ns_host,
                      nameserver_port=ns_port,
                      result_logger=result_logger,
                      min_budget=args.min_budget, max_budget=args.max_budget,
               )
res = bohb.run(n_iterations=args.n_iterations)

# store results
with open(os.path.join(args.shared_directory, 'results.pkl'), 'wb') as fh:
    pickle.dump(res, fh)

# shutdown
bohb.shutdown(shutdown_workers=True)
NS.shutdown()

importing Jupyter notebook from bohb_worker_1to3_11.ipynb
(11, 504960, 2)
(11, 101280, 2)
(11572, 480, 2)
(2321, 480, 2)


DEBUG:hpbandster:wait_for_workers trying to get the condition
DEBUG:hpbandster.run_None.worker.74c293875977.63370:WORKER: Connected to nameserver <Pyro4.core.Proxy at 0x7f061aa82cc0; connected IPv4; for PYRO:Pyro.NameServer@127.0.0.1:43691>
INFO:hpbandster:DISPATCHER: started the 'discover_worker' thread
DEBUG:hpbandster.run_None.worker.74c293875977.63370:WORKER: No dispatcher found. Waiting for one to initiate contact.
INFO:hpbandster.run_None.worker.74c293875977.63370:WORKER: start listening for jobs
INFO:hpbandster:DISPATCHER: started the 'job_runner' thread
INFO:hpbandster:DISPATCHER: Pyro daemon running on 127.0.0.1:33799
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 0 currently in the pool.
INFO:hpbandster:DISPATCHER: discovered new worker, hpbandster.run_None.worker.74c293875977.63370139676796745536
DEBUG:hpbandster:HBMASTER: number of workers changed to 1
DEBUG:hpbandster:Enough workers to start this run!
DEBUG:hp

DEBUG:hpbandster:job_callback for (0, 0, 1) got condition
DEBUG:hpbandster:Only 2 run(s) for budget 66.666667 available, need more than 17 -> can't build model!
DEBUG:hpbandster:HBMASTER: Trying to run another job!
DEBUG:hpbandster:job_callback for (0, 0, 1) finished
DEBUG:hpbandster:start sampling a new configuration.
DEBUG:hpbandster:done sampling a new configuration.
DEBUG:hpbandster:HBMASTER: schedule new run for iteration 0
DEBUG:hpbandster:HBMASTER: trying submitting job (0, 0, 2) to dispatcher
DEBUG:hpbandster:HBMASTER: submitting job (0, 0, 2) to dispatcher
DEBUG:hpbandster:DISPATCHER: trying to submit job (0, 0, 2)
DEBUG:hpbandster:DISPATCHER: trying to notify the job_runner thread.
DEBUG:hpbandster:HBMASTER: job (0, 0, 2) submitted to dispatcher
DEBUG:hpbandster:HBMASTER: running jobs: 1, queue sizes: (0, 1) -> wait
DEBUG:hpbandster:DISPATCHER: Trying to submit another job.
DEBUG:hpbandster:DISPATCHER: starting job (0, 0, 2) on hpbandster.run_None.worker.74c293875977.63370139

DEBUG:hpbandster:DISPATCHER: trying to submit job (1, 0, 0)
DEBUG:hpbandster:DISPATCHER: trying to notify the job_runner thread.
DEBUG:hpbandster:HBMASTER: job (1, 0, 0) submitted to dispatcher
DEBUG:hpbandster:DISPATCHER: Trying to submit another job.
DEBUG:hpbandster:HBMASTER: running jobs: 1, queue sizes: (0, 1) -> wait
DEBUG:hpbandster:DISPATCHER: starting job (1, 0, 0) on hpbandster.run_None.worker.74c293875977.63370139676796745536
DEBUG:hpbandster:DISPATCHER: job (1, 0, 0) dispatched on hpbandster.run_None.worker.74c293875977.63370139676796745536
INFO:hpbandster.run_None.worker.74c293875977.63370:WORKER: start processing job (1, 0, 0)
DEBUG:hpbandster:DISPATCHER: jobs to submit = 0, number of idle workers = 0 -> waiting!
DEBUG:hpbandster.run_None.worker.74c293875977.63370:WORKER: args: ()
DEBUG:hpbandster.run_None.worker.74c293875977.63370:WORKER: kwargs: {'config': {'dropout_rate': 0.34644468243800686, 'lr': 1.4530804190223048e-06, 'lstm_units': 28, 'num_conv_layers': 2, 'num_fc

DEBUG:hpbandster:job_callback for (1, 0, 1) started
DEBUG:hpbandster:DISPATCHER: Trying to submit another job.
DEBUG:hpbandster:DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
DEBUG:hpbandster:job_callback for (1, 0, 1) got condition
DEBUG:hpbandster:Only 3 run(s) for budget 200.000000 available, need more than 17 -> can't build model!
DEBUG:hpbandster:HBMASTER: Trying to run another job!
DEBUG:hpbandster:job_callback for (1, 0, 1) finished
DEBUG:hpbandster:start sampling a new configuration.
DEBUG:hpbandster:done sampling a new configuration.
DEBUG:hpbandster:HBMASTER: schedule new run for iteration 2
DEBUG:hpbandster:HBMASTER: trying submitting job (2, 0, 0) to dispatcher
DEBUG:hpbandster:HBMASTER: submitting job (2, 0, 0) to dispatcher
DEBUG:hpbandster:DISPATCHER: trying to submit job (2, 0, 0)
DEBUG:hpbandster:DISPATCHER: trying to notify the job_runner thread.
DEBUG:hpbandster:HBMASTER: job (2, 0, 0) submitted to dispatcher
DEBUG:hpbandster:DISPATCHER: Tryin

...

DEBUG:hpbandster:DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
DEBUG:hpbandster:done building a new model for budget 200.000000 based on 21/124 split
Best loss for this budget:0.050380





DEBUG:hpbandster:HBMASTER: Trying to run another job!
DEBUG:hpbandster:job_callback for (97, 0, 0) finished
DEBUG:hpbandster:start sampling a new configuration.
DEBUG:hpbandster:best_vector: [0.13659941069953246, 0.8711578281417621, 0.5809902561417216, 0.5057368986168966, 0.652207144784487, 0.24952373444851278, 0.9517522350106645, 1, 0.18333493101227455, 0.8953763725355526, 0.6033355147628701, 0.02095932767190023, 0.9058217903103298, 0.43317238056444674, 0.5704418054799889], 0.007408028029758314, 1.0918912733308108, 0.008088761158283143
DEBUG:hpbandster:done sampling a new configuration.
DEBUG:hpbandster:HBMASTER: schedule new run for iteration 97
DEBUG:hpbandster:HBMASTER: trying submitting job (97, 0, 1) to dispatcher
DEBUG:hpbandster:HBMASTER: submitting job (97, 0, 1) to

DEBUG:hpbandster:DISPATCHER: jobs to submit = 0, number of idle workers = 0 -> waiting!
INFO:hpbandster.run_None.worker.74c293875977.63370:WORKER: start processing job (98, 0, 0)
DEBUG:hpbandster:HBMASTER: running jobs: 1, queue sizes: (0, 1) -> wait
DEBUG:hpbandster.run_None.worker.74c293875977.63370:WORKER: args: ()
DEBUG:hpbandster.run_None.worker.74c293875977.63370:WORKER: kwargs: {'config': {'dropout_rate': 0.7603914536392019, 'lr': 0.042126429760083794, 'lstm_units': 110, 'num_conv_layers': 3, 'num_fc_layers': 3, 'num_fc_units_1': 117, 'num_filters_1': 27, 'optimizer': 'Adam', 'num_fc_units_2': 151, 'num_fc_units_3': 8, 'num_filters_2': 10, 'num_filters_3': 131}, 'budget': 66.66666666666666, 'working_directory': '.'}
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISP

DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster.run_None.worker.74c293875977

DEBUG:hpbandster:job_callback for (98, 0, 2) started
DEBUG:hpbandster:DISPATCHER: Trying to submit another job.
DEBUG:hpbandster:DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
DEBUG:hpbandster:job_callback for (98, 0, 2) got condition
DEBUG:hpbandster:done building a new model for budget 200.000000 based on 22/125 split
Best loss for this budget:0.050380





DEBUG:hpbandster:HBMASTER: Trying to run another job!
DEBUG:hpbandster:job_callback for (98, 0, 2) finished
DEBUG:hpbandster:start sampling a new configuration.
DEBUG:hpbandster:best_vector: [0.10890207809965069, 0.7751116987569672, 0.2482107670920562, 0.36189263734706556, 0.093415897996458, 0.5878313169877744, 0.0532151304785109, 1, 0.5145785312100374, 0.0886242632747618, 0.003249038517770194, 0.8826452780229308, 0.3024024277066866, 0.931934793902055, 0.15544331656246485], 0.006703761659097185, 0.0018014452260322715, 1.2076459437238804e-05
DEBUG:hpbandster:done sampling a new configuration.
DEBUG:hpbandste

DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster.run_None.worker.74c293875977.63370:WORKER: done with job (99, 0, 1), trying to register it.
INFO:hpbandster.run_None.worker.74c293875977.63370:WORKER: registered result for job (99, 0, 1) with dispatcher
DEBUG:hpbandster:DISPATCHER: job (99, 0, 1) finished
DEBUG:hpbandster:DISPATCHER: register_result: lock acquired
DEBUG:hpbandster:DISPATCHER: job (99, 0, 1) on hpbandster.run_None.worker.74c293875977.63370139676796745536 finished
DEBUG:hpbandster:job_id: (99, 0, 1)
kwargs: {'config': {'dropout_rate': 0.6139655438582232, 'lr': 0.00038984608029480353, 'lstm_units': 239, 'num_conv_layers': 5, 'num_fc_layers': 3, 'num_fc_units_1': 177, 'num_filters_1': 32, 'optimizer': 'Adam', 'num_fc_units_2': 15, 'num_fc_units_3': 196, 'num_filters_2': 41, 'num_filters_3': 10, 'num_filters_4': 94, 'num_filters_5': 26}, 'budget': 200.0, 'working_directory': '.'}
result: {'loss': 0.14068442583084106, 'info': {'test accuracy': 0.9123222

In [2]:
# Step 5: Analysis
# Each optimizer returns a hpbandster.core.result.Result object.
# It holds informations about the optimization run like the incumbent (=best) configuration.
# For further details about the Result object, see its documentation.
# Here we simply print out the best config and some statistics about the performed runs.
id2config = res.get_id2config_mapping()
incumbent = res.get_incumbent_id()

print('Best found configuration:', id2config[incumbent]['config'])
print('A total of %i unique configurations where sampled.' % len(id2config.keys()))
print('A total of %i runs where executed.' % len(res.get_all_runs()))
print('Total budget corresponds to %.1f full function evaluations.'%(sum([r.budget for r in res.get_all_runs()])/args.max_budget))

Best found configuration: {'dropout_rate': 0.4877531154485289, 'lr': 0.00016728793369324326, 'lstm_units': 46, 'num_conv_layers': 3, 'num_fc_layers': 2, 'num_fc_units_1': 13, 'num_filters_1': 235, 'optimizer': 'Adam', 'num_fc_units_2': 187, 'num_filters_2': 231, 'num_filters_3': 47}
A total of 250 unique configurations where sampled.
A total of 300 runs where executed.
Total budget corresponds to 200.0 full function evaluations.
